In [63]:
#matrix factorization
import pandas as pd
import numpy as np
import implicit as im
from scipy.sparse import csr_matrix
import random

In [2]:
#read data
proc_dir = "../data/processed"

customers = pd.read_csv(proc_dir+"/customers.csv")
customers = customers['user']
games = pd.read_csv(proc_dir+"/games.csv")
games = games['game']
hours = pd.read_csv(proc_dir+"/hours.csv")
hours = hours[['userIndex', 'gameID', 'hours']]

In [56]:
random.seed(43)

In [87]:
userGameGroup = hours.groupby('userIndex', as_index=False).agg({"hours": "count"})
goodUsers = userGameGroup[userGameGroup.hours > 5]
userSample=[]
for userI in goodUsers['userIndex']:
    userSample.append(userI)
print 'users', len(userSample)
gameGroup = hours.groupby('gameID', as_index=False).agg({"hours": "count"})
goodGames = gameGroup[gameGroup.hours > 150]
gameSample=[]
for gameI in goodGames['gameID']:
    gameSample.append(gameI)
print 'games', len(gameSample)

users 2130
games 75


In [ ]:
# create hours matrix games by customer csr matrix
playM=np.zeros((len(goodGames),len(goodUsers)))
playM.fill(np.nan)

for index, row in hours.iterrows():
    if(row['userIndex'] in userSample and row['gameID'] in gameSample):
        playM[gameSample.index(row['gameID'])][userSample.index(row['userIndex'])]=row['hours']
print playM.shape

In [88]:
playM = csr_matrix(playM)

model = im.als.AlternatingLeastSquares(factors=15)

model.fit(playM)

user_games = playM.T.tocsr()
recommendations = model.recommend(3, user_games,10)

print model.user_factors.shape
print model.item_factors.shape


100%|██████████| 15.0/15 [00:00<00:00, 38.78it/s]

(2130, 15)
(75, 15)
